In [32]:
import configparser
import psycopg2
import pandas as pd

## Connect to Redshift Cluster

In [33]:
config = configparser.ConfigParser()
config.read('dwh.cfg')
S3_LOG_DATA = config.get('S3', 'LOG_DATA')
S3_LOG_JSONPATH = config.get('S3', 'LOG_JSONPATH')
S3_SONG_DATA = config.get('S3', 'SONG_DATA')
DWH_IAM_ROLE_ARN = config.get("IAM_ROLE", "ARN")

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
conn.autocommit = True
cur = conn.cursor()

## Sample Query: Top Users by Songplays

In [ ]:
query = """
SELECT users.user_id,
       users.first_name,
       users.last_name,
       COUNT(DISTINCT songplays.song_id) AS num_songplays
FROM users
LEFT JOIN songsplays ON users.user_id = songplays.user_id
GROUP BY users.user_id,
         users.first_name,
         users.last_name
LIMIT 10
"""

column_names = ['user_id','first_name','last_name','num_songplays']

cur.execute(query)
tuples_list = cur.fetchall()
pd.DataFrame(tuples_list, columns=column_names)

## Sample Query: Top Months by Songplays

In [ ]:
query = """
SELECT times.month,
       times.year,
       COUNT(DISTINCT songplays.song_id) AS num_songplays
FROM times
LEFT JOIN songsplays ON times.start_time = songplays.start_time
GROUP BY times.month,
         times.year
LIMIT 5
"""

column_names = ['month','year','num_songplays']

cur.execute(query)
tuples_list = cur.fetchall()
pd.DataFrame(tuples_list, columns=column_names)

## Sample Query: Top Artists by Songplays

In [ ]:
query = """
SELECT artist.artist_id,
       artist.artist_name,
       COUNT(DISTINCT songplays.song_id) AS num_songplays
FROM artists
LEFT JOIN songsplays ON artist.artist_id = songplays.artist_id
GROUP BY artist.artist_id,
         artist.artist_name
LIMIT 10
"""

column_names = ['artist_id','artist_name','num_songplays']

cur.execute(query)
tuples_list = cur.fetchall()
pd.DataFrame(tuples_list, columns=column_names)

In [ ]:
conn.close()